In [1]:
! pip install requests pandas gspread

In [1]:
from datetime import date, timedelta
import numpy as np
import pandas as pd
from tapdb import TapDBClient
from gsheets import DateSheet
from config import TAPDB_USER, TAPDB_PASS
import asyncio

target_date = (date.today() - timedelta(days=1)).isoformat()
# target_date = '2024-06-28'
if isinstance(target_date, str):
    target_date = date.fromisoformat(target_date)

async def create_tapdb():
    return TapDBClient(TAPDB_USER, TAPDB_PASS, target_date=target_date, tz_offset=1)

async def create_sheet():
    return DateSheet('[RP_DMM] Revenue Report', date=target_date.isoformat())

task_tapdb = asyncio.create_task(create_tapdb())
task_sheet = asyncio.create_task(create_sheet())

await task_tapdb
await task_sheet

tapdb = task_tapdb.result()
sheet = task_sheet.result()


In [2]:
# [RP_DMM] Revenue Report >> 活躍設備
sheet.change_sheet('Daily Re.')
data = tapdb.get_data('RP DMM (PRO)', 'active', dim='os', device=True)

sheet.update_cells('活躍設備\n(tap db)', int(data['DAU_DEVICE'].sum()))
sheet.update_cells('新活躍設備\n(tap db)', int(data['newDAU_DEVICE'].sum()))
data

,key,DAU_DEVICE,newDAU_DEVICE,DAU_DEVICE2,DAU_DEVICE3,DAU_DEVICE7,DAU_DEVICE14,DAU_DEVICE30
key,,,,,,,,
Android,Android,4739,180,4559,4508,4345,4132,3723


In [5]:
# [RP_DMM] Revenue Report >> 轉化率
data = tapdb.get_data('RP DMM (PRO)', 'source', dim='ver')
row = data.loc[max(data.index)]

sheet.change_sheet('轉化率')
sheet.update_cells('版本', row['key'])
sheet.update_cells('新增設備', row['newDevices'])
sheet.update_cells('轉化設備', row['convertedDevices'])

data

,key,newPlayers,totalCharge,totalChargeAmount,firstChargePlayers,firstChargeAmount,newDevices,convertedDevices,devicesConvertRate,dayRetainedRate1,dayRetainedRate7,chargeRate,firstChargeRate,ARPU,ARPPU
key,,,,,,,,,,,,,,,
2.0.0.138,2.0.0.138,117,13,955.26,5,217.98,117,111,94.87,43.59,None,11.11,4.27,1.86,43.6
1.0.0.129,1.0.0.129,0,0,0.00,0,0.00,1,1,100.00,NaN,None,0.00,0.00,0.00,0.0
